# Reference
`https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html`

In [1]:
# Imports
import findspark
findspark.init()
findspark.find()


'/home/prod/spark-3.3.1-bin-hadoop3'

In [2]:
from pyspark import SparkConf
conf = SparkConf()
conf.setAppName('Jupyter Structured Stream PySpark')
conf.setMaster('spark://spark-master:7077');

In [3]:
# Setup spark environment
from pyspark import SparkContext
from pyspark.sql.types import *
from pprint import pprint, pformat
sc = SparkContext.getOrCreate(conf)
print('Spark web UI link: ', sc._jsc.sc().uiWebUrl().get())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/08 14:32:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Spark web UI link:  http://spark-master:4040


## Streaming

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [5]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

dStream = spark.readStream \
  .format("socket")\
  .option("host", "spark-master")\
  .option("port", 9999)\
  .load()

lines = dStream.select(
  explode(
    split(dStream.value, '\n')
  ).alias('text')
)

22/11/08 14:33:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


### Output Merging function

In [6]:
import os

out_file = '/data/test-streaming-merged-output.csv'
cwd = os.getcwd()
data_output_path = cwd + out_file
print('output path: ', data_output_path)
try:
    os.remove(data_output_path)
except Exception as e:
    print(e)


def mergeResult(batchDF, batchID):
    print('batchSize: ', batchDF.count(), 'id: ', batchID)
    batchDF.show()
    masterDf = batchDF.toPandas().to_csv(data_output_path, mode='a', index=False, header=False)

output path:  /home/prod/sparkimental/data/test-streaming-merged-output.csv
[Errno 2] No such file or directory: '/home/prod/sparkimental/data/test-streaming-merged-output.csv'


In [7]:
from time import sleep
import time

query = lines \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName('test-stream-query') \
    .foreachBatch(mergeResult) \
    .start()


def stop_stream_query(query, wait_time):
    """Stop a running streaming query"""
    while query.isActive:
        msg = query.status['message']
        data_avail = query.status['isDataAvailable']
        trigger_active = query.status['isTriggerActive']
        if not data_avail and not trigger_active and msg != "Initializing StreamExecution":
            print('Stopping query...')
            query.stop()
        time.sleep(0.5)

    # Okay wait for the stop to happen
    print('Awaiting termination...')
    query.awaitTermination(wait_time)

stop_stream_query(query, 5000)


22/11/08 14:33:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-532025ec-9c76-4900-b9e2-bd4b51267dc2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/08 14:33:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


batchSize:  0 id:  0
+----+
|text|
+----+
+----+

batchSize:  35 id:  1
+--------------------+
|                text|
+--------------------+
|Animal Crossing; ...|
|With a game this ...|
|Above all else, A...|
|Nintendo's comfor...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|Animal Crossing: ...|
|I always knew Ani...|
|Animal Crossing: ...|
|The amazing, rela...|
|Animal Crossing: ...|
|New Horizons has ...|
|Animal Crossing: ...|
|Nintendo Switch g...|
|Animal Crossing i...|
|Animal Crossing f...|
|Know that if you’...|
|Animal Crossing: ...|
+--------------------+
only showing top 20 rows

batchSize:  7 id:  2
+--------------------+
|                text|
+--------------------+
|Animal Crossing: ...|
|Animal Crossing N...|
|Animal Crossing r...|
|Animal Crossing: ...|
|There continues t...|
|Under the illusio...|
|The series has ma...|
+--------------------+

batchSize:  4 id:  3
+--------------------+
|                text|
+-------

In [8]:
import pandas as pd

df_merged = pd.read_csv('.'+out_file, header=None)
df_init = pd.read_csv('./data/animal-crossing.csv')


In [9]:
print(df_merged.shape, df_init.shape)
print('Valid stream? ', df_merged.shape[0] == df_init.shape[0])

(107, 1) (107, 4)
Valid stream?  True
22/11/08 14:41:43 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.56.111: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/11/08 14:41:45 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/11/08 14:41:45 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$Clien